In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
from tensorflow.keras.models import load_model
import albumentations as A

# Custom objects required for loading the U-Net segmentation models
import sys 
sys.path.append('..')
from segmentation_losses import (
    dice_coefficient,
    dice_loss,
    log_cosh_dice_loss,
    iou
)

In [2]:
model = load_model(
    "../trained_models/task03_liver_unet_3d_segmentation_model.h5",
    custom_objects={
        "log_cosh_dice_loss": log_cosh_dice_loss,
        "dice_coefficient": dice_coefficient,
        "iou": iou
    }
)

In [3]:
def standardize(mri):
    """
    Standardize mean and standard deviation of each channel and z_dimension slice to mean 0 and standard 
     deviation 1.

    Note: setting the type of the input mri to np.float16 beforehand causes issues, set it afterwards.

    Args:
        mri (np.array): input mri, shape (dim_x, dim_y, dim_z, num_channels)
    Returns:
        standardized_mri (np.array): standardized version of input mri
    """

    standardized_mri = np.zeros(mri.shape)

    # Iterate over channels
    for c in range(mri.shape[3]):
        # Iterate over the `z` depth dimension
        for z in range(mri.shape[2]):
            # Get a slice of the mri at channel c and z-th dimension
            mri_slice = mri[:, :, z, c]

            # Subtract the mean from mri_slice
            centered = mri_slice - np.mean(mri_slice)

            # Divide by the standard deviation (only if it is different from zero)
            if np.std(centered) != 0:
                centered_scaled = centered / np.std(centered)

                # Update the slice of standardized mri with the centered and scaled mri
                standardized_mri[:, :, z, c] = centered_scaled

    return standardized_mri

### 1. Visualizing predictions on validation set images

### 1.1. Visualizing prediction on liver_118 file

### 1.1.1 Chunk 1

In [4]:
mri_chunk1 = nib.load("../datasets/Task03_Liver_320x320x96/val/images/liver_118_0.nii.gz").get_fdata().astype(np.float32)

In [5]:
mask_chunk1 = nib.load("../datasets/Task03_Liver_320x320x96/val/masks/liver_118_0.nii.gz").get_fdata().astype(np.uint8)

In [6]:
mask_chunk1.shape

(320, 320, 96)

In [7]:
mri_standardized_chunk1 = standardize(mri_chunk1.reshape((mri_chunk1.shape[0], mri_chunk1.shape[1], mri_chunk1.shape[2], 1)))

In [8]:
mri_standardized_chunk1 = np.expand_dims(mri_standardized_chunk1, axis=0)  # Keras models require an additional dimension of 'batch_size'

In [9]:
prediction_chunk1 = model.predict(mri_standardized_chunk1)

In [10]:
prediction_chunk1 = np.squeeze(prediction_chunk1, axis=0)  # Remove 'batch_size' dimension

In [11]:
prediction_chunk1.shape

(320, 320, 96, 3)

In [12]:
prediction_chunk1

array([[[[9.9144709e-01, 1.0611064e-03, 7.4918293e-03],
         [9.9168903e-01, 9.6662482e-04, 7.3443474e-03],
         [9.9132174e-01, 1.1117188e-03, 7.5665936e-03],
         ...,
         [9.9072623e-01, 1.3668649e-03, 7.9069659e-03],
         [9.8983788e-01, 1.4262591e-03, 8.7358961e-03],
         [9.8372722e-01, 2.6191953e-03, 1.3653597e-02]],

        [[9.9112731e-01, 1.1923724e-03, 7.6803491e-03],
         [9.9227810e-01, 7.5490907e-04, 6.9670053e-03],
         [9.9156076e-01, 1.0161562e-03, 7.4230190e-03],
         ...,
         [9.9208647e-01, 8.2083273e-04, 7.0926202e-03],
         [9.9231213e-01, 7.4351119e-04, 6.9444147e-03],
         [9.9237663e-01, 7.2214176e-04, 6.9013098e-03]],

        [[9.9140209e-01, 1.0791222e-03, 7.5187590e-03],
         [9.9235487e-01, 7.2929176e-04, 6.9158445e-03],
         [9.9172157e-01, 9.5423282e-04, 7.3241689e-03],
         ...,
         [9.9176019e-01, 9.3965360e-04, 7.3001618e-03],
         [9.9129468e-01, 1.1227608e-03, 7.5825467e-03],
  

In [13]:
type(prediction_chunk1)

numpy.ndarray

In [14]:
prediction_chunk1 = np.argmax(prediction_chunk1, axis=3)  # Apply Softmax

In [15]:
prediction_chunk1.shape

(320, 320, 96)

In [16]:
prediction_chunk1 = prediction_chunk1.reshape((prediction_chunk1.shape[0], prediction_chunk1.shape[1], prediction_chunk1.shape[2]))

In [17]:
prediction_chunk1.shape

(320, 320, 96)

In [18]:
np.unique(prediction_chunk1)

array([0, 2])

#### Model prediction vs groundtruth number of labels

In [19]:
# Number of labels per class in model prediction
print("Background - class 0")
print(len(prediction_chunk1[prediction_chunk1 == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(prediction_chunk1[prediction_chunk1 == 1]))
print("--------")
print("Cancer - class 2")
print(len(prediction_chunk1[prediction_chunk1 == 2]))

Background - class 0
9830389
--------
Left Atrium - class 1
0
--------
Cancer - class 2
11


In [20]:
# Number of labels per class in ground truth mask
print("Background - class 0")
print(len(mask_chunk1[mask_chunk1 == 0])) 
print("--------")
print("Liver - class 1")
print(len(mask_chunk1[mask_chunk1 == 1]))
print("--------")
print("Cancer - class 2")
print(len(mask_chunk1[mask_chunk1 == 2]))

Background - class 0
9830400
--------
Liver - class 1
0
--------
Cancer - class 2
0


In [21]:
classes_dict = {
    'Background': 0,
    'Liver': 1,
    'Cancer': 2
}

# Create button values
select_class = ToggleButtons(
    options=['Background', 'Liver', 'Cancer', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=mri_chunk1.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer):
    print(f"Plotting Layer: {layer} | Label: {seg_class}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 3, 1)
    plt.title("Liver_118", fontsize=20)
    plt.imshow(mri_chunk1[:, :, layer], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 3, 2)
    if seg_class == "All":
        mask_groundtruth = mask_chunk1[:, :, layer]
        plt.title("Groundtruth mask", fontsize=20)
        plt.imshow(mask_groundtruth)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_groundtruth = np.where(mask_chunk1[:, :, layer] == img_label, 255, 0)
        plt.title("Groundtruth mask", fontsize=20)
        plt.imshow(mask_groundtruth, cmap='gray')
        plt.axis('off');
    
    fig.add_subplot(1, 3, 3)
    if seg_class == "All":
        mask_model_prediction = prediction_chunk1[:, :, layer]
        plt.title("Model prediction mask", fontsize=20)
        plt.imshow(mask_model_prediction)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_model_prediction = np.where(prediction_chunk1[:, :, layer] == img_label, 255, 0)
        plt.title("Model prediction mask", fontsize=20)
        plt.imshow(mask_model_prediction, cmap='gray')
        plt.axis('off');

    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer)

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### 1.1.2 Chunk 2

In [22]:
mri_chunk2 = nib.load("../datasets/Task03_Liver_320x320x96/val/images/liver_118_1.nii.gz").get_fdata().astype(np.float32)

In [23]:
mask_chunk2 = nib.load("../datasets/Task03_Liver_320x320x96/val/masks/liver_118_1.nii.gz").get_fdata().astype(np.uint8)

In [24]:
mask_chunk2.shape

(320, 320, 96)

In [25]:
mri_standardized_chunk2 = standardize(mri_chunk2.reshape((mri_chunk2.shape[0], mri_chunk2.shape[1], mri_chunk2.shape[2], 1)))

In [26]:
mri_standardized_chunk2 = np.expand_dims(mri_standardized_chunk2, axis=0)  # Keras models require an additional dimension of 'batch_size'

In [27]:
prediction_chunk2 = model.predict(mri_standardized_chunk2)

In [28]:
prediction_chunk2 = np.squeeze(prediction_chunk2, axis=0)  # Remove 'batch_size' dimension

In [29]:
prediction_chunk2.shape

(320, 320, 96, 3)

In [30]:
prediction_chunk2

array([[[[9.9127227e-01, 1.1319756e-03, 7.5957645e-03],
         [9.9151659e-01, 1.0335282e-03, 7.4498956e-03],
         [9.9091280e-01, 1.2843269e-03, 7.8028231e-03],
         ...,
         [9.9081087e-01, 1.3291246e-03, 7.8599844e-03],
         [9.9119979e-01, 1.1620121e-03, 7.6382654e-03],
         [9.8541206e-01, 2.2904917e-03, 1.2297538e-02]],

        [[9.9054396e-01, 1.4495545e-03, 8.0064097e-03],
         [9.9144953e-01, 1.0601139e-03, 7.4903369e-03],
         [9.9075621e-01, 1.3534444e-03, 7.8903800e-03],
         ...,
         [9.9185151e-01, 9.0562704e-04, 7.2429581e-03],
         [9.9216264e-01, 7.9428387e-04, 7.0430245e-03],
         [9.9236917e-01, 7.2454941e-04, 6.9062160e-03]],

        [[9.9101549e-01, 1.2399430e-03, 7.7446094e-03],
         [9.9214339e-01, 8.0099062e-04, 7.0556779e-03],
         [9.9170834e-01, 9.5928035e-04, 7.3324153e-03],
         ...,
         [9.9200165e-01, 8.5094594e-04, 7.1473578e-03],
         [9.9116862e-01, 1.1749965e-03, 7.6563666e-03],
  

In [31]:
type(prediction_chunk2)

numpy.ndarray

In [32]:
prediction_chunk2 = np.argmax(prediction_chunk2, axis=3)  # Apply Softmax

In [33]:
prediction_chunk2.shape

(320, 320, 96)

In [34]:
prediction_chunk2 = prediction_chunk2.reshape((prediction_chunk2.shape[0], prediction_chunk2.shape[1], prediction_chunk2.shape[2]))

In [35]:
prediction_chunk2.shape

(320, 320, 96)

In [36]:
np.unique(prediction_chunk2)

array([0])

#### Model prediction vs groundtruth number of labels

In [37]:
# Number of labels per class in model prediction
print("Background - class 0")
print(len(prediction_chunk2[prediction_chunk2 == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(prediction_chunk2[prediction_chunk2 == 1]))
print("--------")
print("Cancer - class 2")
print(len(prediction_chunk2[prediction_chunk2 == 2]))

Background - class 0
9830400
--------
Left Atrium - class 1
0
--------
Cancer - class 2
0


In [38]:
# Number of labels per class in ground truth mask
print("Background - class 0")
print(len(mask_chunk2[mask_chunk2 == 0])) 
print("--------")
print("Liver - class 1")
print(len(mask_chunk2[mask_chunk2 == 1]))
print("--------")
print("Cancer - class 2")
print(len(mask_chunk2[mask_chunk2 == 2]))

Background - class 0
9830400
--------
Liver - class 1
0
--------
Cancer - class 2
0


In [39]:
classes_dict = {
    'Background': 0,
    'Liver': 1,
    'Cancer': 2
}

# Create button values
select_class = ToggleButtons(
    options=['Background', 'Liver', 'Cancer', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=mri_chunk2.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer):
    print(f"Plotting Layer: {layer} | Label: {seg_class}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 3, 1)
    plt.title("Liver_118", fontsize=20)
    plt.imshow(mri_chunk2[:, :, layer], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 3, 2)
    if seg_class == "All":
        mask_groundtruth = mask_chunk2[:, :, layer]
        plt.title("Groundtruth mask", fontsize=20)
        plt.imshow(mask_groundtruth)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_groundtruth = np.where(mask_chunk2[:, :, layer] == img_label, 255, 0)
        plt.title("Groundtruth mask", fontsize=20)
        plt.imshow(mask_groundtruth, cmap='gray')
        plt.axis('off');
    
    fig.add_subplot(1, 3, 3)
    if seg_class == "All":
        mask_model_prediction = prediction_chunk2[:, :, layer]
        plt.title("Model prediction mask", fontsize=20)
        plt.imshow(mask_model_prediction)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_model_prediction = np.where(prediction_chunk2[:, :, layer] == img_label, 255, 0)
        plt.title("Model prediction mask", fontsize=20)
        plt.imshow(mask_model_prediction, cmap='gray')
        plt.axis('off');

    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer)

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### 1.1.3 Chunk 3

In [40]:
mri_chunk3 = nib.load("../datasets/Task03_Liver_320x320x96/val/images/liver_118_2.nii.gz").get_fdata().astype(np.float32)

In [41]:
mask_chunk3 = nib.load("../datasets/Task03_Liver_320x320x96/val/masks/liver_118_2.nii.gz").get_fdata().astype(np.uint8)

In [42]:
mask_chunk3.shape

(320, 320, 96)

In [43]:
mri_standardized_chunk3 = standardize(mri_chunk3.reshape((mri_chunk3.shape[0], mri_chunk3.shape[1], mri_chunk3.shape[2], 1)))

In [44]:
mri_standardized_chunk3 = np.expand_dims(mri_standardized_chunk3, axis=0)  # Keras models require an additional dimension of 'batch_size'

In [45]:
prediction_chunk3 = model.predict(mri_standardized_chunk3)

In [46]:
prediction_chunk3 = np.squeeze(prediction_chunk3, axis=0)  # Remove 'batch_size' dimension

In [47]:
prediction_chunk3.shape

(320, 320, 96, 3)

In [48]:
prediction_chunk3

array([[[[9.9128175e-01, 1.1280631e-03, 7.5901635e-03],
         [9.9146515e-01, 1.0539290e-03, 7.4809990e-03],
         [9.9051183e-01, 1.4643791e-03, 8.0237612e-03],
         ...,
         [9.9079520e-01, 1.3361225e-03, 7.8687780e-03],
         [9.8741084e-01, 1.8526757e-03, 1.0736396e-02],
         [9.8203146e-01, 2.9052941e-03, 1.5063229e-02]],

        [[9.9027842e-01, 1.5738792e-03, 8.1477733e-03],
         [9.9106032e-01, 1.2207885e-03, 7.7189701e-03],
         [9.9021190e-01, 1.6056562e-03, 8.1824716e-03],
         ...,
         [9.9230897e-01, 7.4453198e-04, 6.9464459e-03],
         [9.9268156e-01, 6.2555441e-04, 6.6928980e-03],
         [9.9260932e-01, 6.4777629e-04, 6.7429901e-03]],

        [[9.9088717e-01, 1.2955298e-03, 7.8172646e-03],
         [9.9201655e-01, 8.4565085e-04, 7.1378471e-03],
         [9.9174106e-01, 9.4686658e-04, 7.3120748e-03],
         ...,
         [9.9164951e-01, 9.8177686e-04, 7.3687471e-03],
         [9.9165219e-01, 9.8071829e-04, 7.3670489e-03],
  

In [49]:
type(prediction_chunk3)

numpy.ndarray

In [50]:
prediction_chunk3 = np.argmax(prediction_chunk3, axis=3)  # Apply Softmax

In [51]:
prediction_chunk3.shape

(320, 320, 96)

In [52]:
prediction_chunk3 = prediction_chunk3.reshape((prediction_chunk3.shape[0], prediction_chunk3.shape[1], prediction_chunk3.shape[2]))

In [53]:
prediction_chunk3.shape

(320, 320, 96)

In [54]:
np.unique(prediction_chunk3)

array([0, 1, 2])

#### Model prediction vs groundtruth number of labels

In [55]:
# Number of labels per class in model prediction
print("Background - class 0")
print(len(prediction_chunk3[prediction_chunk3 == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(prediction_chunk3[prediction_chunk3 == 1]))
print("--------")
print("Cancer - class 2")
print(len(prediction_chunk3[prediction_chunk3 == 2]))

Background - class 0
9100165
--------
Left Atrium - class 1
423769
--------
Cancer - class 2
306466


In [56]:
# Number of labels per class in ground truth mask
print("Background - class 0")
print(len(mask_chunk3[mask_chunk3 == 0])) 
print("--------")
print("Liver - class 1")
print(len(mask_chunk3[mask_chunk3 == 1]))
print("--------")
print("Cancer - class 2")
print(len(mask_chunk3[mask_chunk3 == 2]))

Background - class 0
9040579
--------
Liver - class 1
626705
--------
Cancer - class 2
163116


In [57]:
classes_dict = {
    'Background': 0,
    'Liver': 1,
    'Cancer': 2
}

# Create button values
select_class = ToggleButtons(
    options=['Background', 'Liver', 'Cancer', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=mri_chunk3.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer):
    print(f"Plotting Layer: {layer} | Label: {seg_class}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 3, 1)
    plt.title("Liver_118", fontsize=20)
    plt.imshow(mri_chunk3[:, :, layer], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 3, 2)
    if seg_class == "All":
        mask_groundtruth = mask_chunk3[:, :, layer]
        plt.title("Groundtruth mask", fontsize=20)
        plt.imshow(mask_groundtruth)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_groundtruth = np.where(mask_chunk3[:, :, layer] == img_label, 255, 0)
        plt.title("Groundtruth mask", fontsize=20)
        plt.imshow(mask_groundtruth, cmap='gray')
        plt.axis('off');
    
    fig.add_subplot(1, 3, 3)
    if seg_class == "All":
        mask_model_prediction = prediction_chunk3[:, :, layer]
        plt.title("Model prediction mask", fontsize=20)
        plt.imshow(mask_model_prediction)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_model_prediction = np.where(prediction_chunk3[:, :, layer] == img_label, 255, 0)
        plt.title("Model prediction mask", fontsize=20)
        plt.imshow(mask_model_prediction, cmap='gray')
        plt.axis('off');

    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer)

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### 1.1.4 Chunk 4

In [58]:
mri_chunk4 = nib.load("../datasets/Task03_Liver_320x320x96/val/images/liver_118_3.nii.gz").get_fdata().astype(np.float32)

In [59]:
mask_chunk4 = nib.load("../datasets/Task03_Liver_320x320x96/val/masks/liver_118_3.nii.gz").get_fdata().astype(np.uint8)

In [60]:
mask_chunk4.shape

(320, 320, 96)

In [61]:
mri_standardized_chunk4 = standardize(mri_chunk4.reshape((mri_chunk4.shape[0], mri_chunk4.shape[1], mri_chunk4.shape[2], 1)))

In [62]:
mri_standardized_chunk4 = np.expand_dims(mri_standardized_chunk4, axis=0)  # Keras models require an additional dimension of 'batch_size'

In [63]:
prediction_chunk4 = model.predict(mri_standardized_chunk4)

In [64]:
prediction_chunk4 = np.squeeze(prediction_chunk4, axis=0)  # Remove 'batch_size' dimension

In [65]:
prediction_chunk4.shape

(320, 320, 96, 3)

In [66]:
prediction_chunk4

array([[[[9.9140304e-01, 1.0787579e-03, 7.5182170e-03],
         [9.9164128e-01, 9.8489353e-04, 7.3737297e-03],
         [9.9124289e-01, 1.1441081e-03, 7.6130382e-03],
         ...,
         [9.9001384e-01, 1.3936443e-03, 8.5925050e-03],
         [9.8234069e-01, 2.6660813e-03, 1.4993212e-02],
         [9.8027575e-01, 3.1082125e-03, 1.6616000e-02]],

        [[9.9100786e-01, 1.2432025e-03, 7.7489377e-03],
         [9.9210268e-01, 8.1518100e-04, 7.0821722e-03],
         [9.9139470e-01, 1.0821259e-03, 7.5232172e-03],
         ...,
         [9.9276090e-01, 6.0167717e-04, 6.6374848e-03],
         [9.9352080e-01, 3.9916806e-04, 6.0800272e-03],
         [9.9175042e-01, 6.4937904e-04, 7.6001128e-03]],

        [[9.9129301e-01, 1.1234209e-03, 7.5834920e-03],
         [9.9230158e-01, 7.4701710e-04, 6.9513903e-03],
         [9.9169630e-01, 9.6387218e-04, 7.3398855e-03],
         ...,
         [9.9169695e-01, 9.6359436e-04, 7.3394324e-03],
         [9.9244398e-01, 7.0011953e-04, 6.8558250e-03],
  

In [67]:
type(prediction_chunk4)

numpy.ndarray

In [68]:
prediction_chunk4 = np.argmax(prediction_chunk4, axis=3)  # Apply Softmax

In [69]:
prediction_chunk4.shape

(320, 320, 96)

In [70]:
prediction_chunk4 = prediction_chunk4.reshape((prediction_chunk4.shape[0], prediction_chunk4.shape[1], prediction_chunk4.shape[2]))

In [71]:
prediction_chunk4.shape

(320, 320, 96)

In [72]:
np.unique(prediction_chunk4)

array([0, 1, 2])

#### Model prediction vs groundtruth number of labels

In [73]:
# Number of labels per class in model prediction
print("Background - class 0")
print(len(prediction_chunk4[prediction_chunk4 == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(prediction_chunk4[prediction_chunk4 == 1]))
print("--------")
print("Cancer - class 2")
print(len(prediction_chunk4[prediction_chunk4 == 2]))

Background - class 0
9651229
--------
Left Atrium - class 1
178389
--------
Cancer - class 2
782


In [74]:
# Number of labels per class in ground truth mask
print("Background - class 0")
print(len(mask_chunk4[mask_chunk4 == 0])) 
print("--------")
print("Liver - class 1")
print(len(mask_chunk4[mask_chunk4 == 1]))
print("--------")
print("Cancer - class 2")
print(len(mask_chunk4[mask_chunk4 == 2]))

Background - class 0
9622030
--------
Liver - class 1
208073
--------
Cancer - class 2
297


In [75]:
classes_dict = {
    'Background': 0,
    'Liver': 1,
    'Cancer': 2
}

# Create button values
select_class = ToggleButtons(
    options=['Background', 'Liver', 'Cancer', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=mri_chunk4.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer):
    print(f"Plotting Layer: {layer} | Label: {seg_class}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 3, 1)
    plt.title("Liver_118", fontsize=20)
    plt.imshow(mri_chunk4[:, :, layer], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 3, 2)
    if seg_class == "All":
        mask_groundtruth = mask_chunk4[:, :, layer]
        plt.title("Groundtruth mask", fontsize=20)
        plt.imshow(mask_groundtruth)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_groundtruth = np.where(mask_chunk4[:, :, layer] == img_label, 255, 0)
        plt.title("Groundtruth mask", fontsize=20)
        plt.imshow(mask_groundtruth, cmap='gray')
        plt.axis('off');
    
    fig.add_subplot(1, 3, 3)
    if seg_class == "All":
        mask_model_prediction = prediction_chunk4[:, :, layer]
        plt.title("Model prediction mask", fontsize=20)
        plt.imshow(mask_model_prediction)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_model_prediction = np.where(prediction_chunk4[:, :, layer] == img_label, 255, 0)
        plt.title("Model prediction mask", fontsize=20)
        plt.imshow(mask_model_prediction, cmap='gray')
        plt.axis('off');

    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer)

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### 1.1.5 Chunk 5

In [76]:
mri_chunk5 = nib.load("../datasets/Task03_Liver_320x320x96/val/images/liver_118_4.nii.gz").get_fdata().astype(np.float32)

In [77]:
mask_chunk5 = nib.load("../datasets/Task03_Liver_320x320x96/val/masks/liver_118_4.nii.gz").get_fdata().astype(np.uint8)

In [78]:
mask_chunk5.shape

(320, 320, 96)

In [79]:
mri_standardized_chunk5 = standardize(mri_chunk5.reshape((mri_chunk5.shape[0], mri_chunk5.shape[1], mri_chunk5.shape[2], 1)))

In [80]:
mri_standardized_chunk5 = np.expand_dims(mri_standardized_chunk5, axis=0)  # Keras models require an additional dimension of 'batch_size'

In [81]:
prediction_chunk5 = model.predict(mri_standardized_chunk5)

In [82]:
prediction_chunk5 = np.squeeze(prediction_chunk5, axis=0)  # Remove 'batch_size' dimension

In [83]:
prediction_chunk5.shape

(320, 320, 96, 3)

In [84]:
prediction_chunk5

array([[[[9.9155468e-01, 1.0185825e-03, 7.4267974e-03],
         [9.9170083e-01, 9.6211693e-04, 7.3370324e-03],
         [9.9133754e-01, 1.1052605e-03, 7.5572059e-03],
         ...,
         [9.8979098e-01, 1.8128697e-03, 8.3962660e-03],
         [9.9071807e-01, 1.3705229e-03, 7.9114633e-03],
         [9.9073845e-01, 1.3613931e-03, 7.9002185e-03]],

        [[9.9160117e-01, 1.0004540e-03, 7.3984163e-03],
         [9.9274868e-01, 6.0529297e-04, 6.6459822e-03],
         [9.9203098e-01, 8.4048527e-04, 7.1285185e-03],
         ...,
         [9.9188274e-01, 8.9409249e-04, 7.2231772e-03],
         [9.9205106e-01, 8.3337619e-04, 7.1156113e-03],
         [9.9195671e-01, 8.6711097e-04, 7.1761124e-03]],

        [[9.9158722e-01, 1.0058417e-03, 7.4068932e-03],
         [9.9249023e-01, 6.8524608e-04, 6.8244603e-03],
         [9.9186534e-01, 9.0048270e-04, 7.2341608e-03],
         ...,
         [9.8885053e-01, 2.3100947e-03, 8.8393353e-03],
         [9.9096620e-01, 1.2611294e-03, 7.7725952e-03],
  

In [85]:
type(prediction_chunk5)

numpy.ndarray

In [86]:
prediction_chunk5 = np.argmax(prediction_chunk5, axis=3)  # Apply Softmax

In [87]:
prediction_chunk5.shape

(320, 320, 96)

In [88]:
prediction_chunk5 = prediction_chunk5.reshape((prediction_chunk5.shape[0], prediction_chunk5.shape[1], prediction_chunk5.shape[2]))

In [89]:
prediction_chunk5.shape

(320, 320, 96)

In [90]:
np.unique(prediction_chunk5)

array([0, 1, 2])

#### Model prediction vs groundtruth number of labels

In [91]:
# Number of labels per class in model prediction
print("Background - class 0")
print(len(prediction_chunk5[prediction_chunk5 == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(prediction_chunk5[prediction_chunk5 == 1]))
print("--------")
print("Cancer - class 2")
print(len(prediction_chunk5[prediction_chunk5 == 2]))

Background - class 0
9794398
--------
Left Atrium - class 1
25053
--------
Cancer - class 2
10949


In [92]:
# Number of labels per class in ground truth mask
print("Background - class 0")
print(len(mask_chunk5[mask_chunk5 == 0])) 
print("--------")
print("Liver - class 1")
print(len(mask_chunk5[mask_chunk5 == 1]))
print("--------")
print("Cancer - class 2")
print(len(mask_chunk5[mask_chunk5 == 2]))

Background - class 0
9830400
--------
Liver - class 1
0
--------
Cancer - class 2
0


In [93]:
classes_dict = {
    'Background': 0,
    'Liver': 1,
    'Cancer': 2
}

# Create button values
select_class = ToggleButtons(
    options=['Background', 'Liver', 'Cancer', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=mri_chunk5.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer):
    print(f"Plotting Layer: {layer} | Label: {seg_class}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 3, 1)
    plt.title("Liver_118", fontsize=20)
    plt.imshow(mri_chunk5[:, :, layer], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 3, 2)
    if seg_class == "All":
        mask_groundtruth = mask_chunk5[:, :, layer]
        plt.title("Groundtruth mask", fontsize=20)
        plt.imshow(mask_groundtruth)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_groundtruth = np.where(mask_chunk5[:, :, layer] == img_label, 255, 0)
        plt.title("Groundtruth mask", fontsize=20)
        plt.imshow(mask_groundtruth, cmap='gray')
        plt.axis('off');
    
    fig.add_subplot(1, 3, 3)
    if seg_class == "All":
        mask_model_prediction = prediction_chunk5[:, :, layer]
        plt.title("Model prediction mask", fontsize=20)
        plt.imshow(mask_model_prediction)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_model_prediction = np.where(prediction_chunk5[:, :, layer] == img_label, 255, 0)
        plt.title("Model prediction mask", fontsize=20)
        plt.imshow(mask_model_prediction, cmap='gray')
        plt.axis('off');

    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer)

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### 2. Testing on the medical decathlon test images for Task 3

In [94]:
def generate_model_prediction(model, mri):
    mri_standardized = standardize(mri)
    mri_standardized = np.expand_dims(mri_standardized, axis=0)  # Keras models require an additional dimension of 'batch_size'

    prediction = model.predict(mri_standardized)
    prediction = np.squeeze(prediction, axis=0)  # Remove 'batch_size' dimension

    prediction = np.argmax(prediction, axis=3)  # Apply Softmax
    prediction = prediction.reshape(prediction.shape[0], prediction.shape[1], prediction.shape[2])  # Remove extra channel dimension

    return prediction

In [95]:
def generate_model_prediction_full_file_chunks(model, mri):
    file_layer_size = 96
    original_mri_image_height_size = mri.shape[0]
    original_mri_image_width_size = mri.shape[1]
    original_file_layer_size = mri.shape[2]

    num_chunks = int(np.ceil(original_file_layer_size / file_layer_size))
    prediction = np.zeros((original_mri_image_height_size, original_mri_image_width_size, original_file_layer_size))
    chunks = np.zeros((num_chunks, original_mri_image_height_size, original_mri_image_width_size, file_layer_size, 1))

    reshaped_mri = mri.reshape((original_mri_image_height_size, original_mri_image_width_size, original_file_layer_size, 1))

    # Split the file into chunks
    for i in range(num_chunks):
        if i == num_chunks - 1:  # last iteration (padding the last chunk with zeros and using it)
            chunks[i, :, :, :original_file_layer_size % file_layer_size, :] = reshaped_mri[:, :, i*file_layer_size: i*file_layer_size + (original_file_layer_size % file_layer_size), :]
        else:
            chunks[i, :, :, :, :] = reshaped_mri[:, :, i*file_layer_size: (i+1)*file_layer_size, :]

    # Generate predictions per chunk
    for i in range(num_chunks):
        if i == num_chunks - 1:  # Last chunk, only get the remainder of the layers
            prediction[:, :, i*file_layer_size:] = generate_model_prediction(model=model, mri=chunks[i, :, :, :])[:, :, :original_file_layer_size - (i*file_layer_size)]  # Returns type np.uint8
        else:
            prediction[:, :, i*file_layer_size: (i+1)*file_layer_size] = generate_model_prediction(model=model, mri=chunks[i, :, :, :])  # Returns type np.uint8
    
    return prediction

### 2.1 Visualizing prediction on liver_132

In [96]:
mri = nib.load("../datasets/Task03_Liver/test_images_for_model_prediction_submission/liver_132.nii.gz").get_fdata().astype(np.float32)

In [97]:
mri.shape

(512, 512, 163)

In [98]:
transform = A.Compose(
    p=1.0,
    transforms=[
        A.Resize(
            height=320,
            width=320,
            interpolation=1,
            p=1
        )
    ]
)

transform_original_dimensions = A.Compose(
    p=1.0,
    transforms=[
        A.Resize(
            height=mri.shape[0],
            width=mri.shape[1],
            interpolation=1,
            p=1
        )
    ]
)

In [99]:
# Case 1: If there are more than one channel, Albumentations.Resize() would only work on each
#  channel separately, working on all dimensions at once would cause it to fail
if len(mri.shape) >= 4:
    temp_mri_channels = np.zeros((image_height_size, image_width_size, original_file_layer_size, mri.shape[3]))
    for i in range(int(mri.shape[3])):
        transformed = transform(image=mri[:, :, :, i])
        temp_mri = transformed["image"]
        temp_mri_channels[:, :, :, i] = temp_mri[:, :, :]

    mri = temp_mri_channels
else:
    # Case 2: One channel file, Albumentations.Resize() would work normally
    transformed = transform(image=mri)
    mri = transformed["image"]

In [100]:
mri.shape

(320, 320, 163)

In [101]:
prediction = generate_model_prediction_full_file_chunks(model, mri)

In [102]:
prediction.shape

(320, 320, 163)

In [103]:
type(prediction)

numpy.ndarray

In [104]:
prediction.shape

(320, 320, 163)

In [105]:
np.unique(prediction)

array([0., 1., 2.])

#### Model prediction number of labels (no available masks)

In [106]:
# Number of labels per class in model prediction
print("Background - class 0")
print(len(prediction[prediction == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(prediction[prediction == 1]))
print("--------")
print("Cancer - class 2")
print(len(prediction[prediction == 2]))

Background - class 0
16336922
--------
Left Atrium - class 1
346794
--------
Cancer - class 2
7484


In [107]:
classes_dict = {
    'Background': 0,
    'Liver': 1,
    'Cancer': 2
}

# Create button values
select_class = ToggleButtons(
    options=['Background', 'Liver', 'Cancer', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=mri.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer):
    print(f"Plotting Layer: {layer} | Label: {seg_class}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.title("liver_132", fontsize=20)
    plt.imshow(mri[:, :, layer], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        maskmodel_prediction = prediction[:, :, layer]
        plt.title("Model prediction mask", fontsize=20)
        plt.imshow(maskmodel_prediction)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_model_prediction = np.where(prediction[:, :, layer] == img_label, 255, 0)
        plt.title("Model prediction mask", fontsize=20)
        plt.imshow(mask_model_prediction, cmap='gray')
        plt.axis('off');

    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer)

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

#### Resizing to original size

In [108]:
prediction = prediction.astype(np.uint8)

# Transforming both image and mask at the same time yields different amount of labels
#  than doing them separately
transformed_mri = transform_original_dimensions(image=mri)
transformed_prediction = transform_original_dimensions(image=prediction)

original_size_mri = transformed_mri["image"].astype(np.float32)
original_size_prediction = transformed_prediction["image"].astype(np.uint8)

In [109]:
original_size_mri.shape

(512, 512, 163)

In [110]:
original_size_prediction.shape

(512, 512, 163)

In [111]:
# Number of labels per class in model prediction
print("Background - class 0")
print(len(original_size_prediction[original_size_prediction == 0])) 
print("--------")
print("Left Atrium - class 1")
print(len(original_size_prediction[original_size_prediction == 1]))
print("--------")
print("Cancer - class 2")
print(len(original_size_prediction[original_size_prediction == 2]))

Background - class 0
41824947
--------
Left Atrium - class 1
888821
--------
Cancer - class 2
15704


In [113]:
classes_dict = {
    'Background': 0,
    'Liver': 1,
    'Cancer': 2
}

# Create button values
select_class = ToggleButtons(
    options=['Background', 'Liver', 'Cancer', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=original_size_mri.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer):
    print(f"Plotting Layer: {layer} | Label: {seg_class}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.title("liver_132", fontsize=20)
    plt.imshow(original_size_mri[:, :, layer], cmap='gray');
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        mask_model_prediction = original_size_prediction[:, :, layer]
        plt.title("Model prediction mask", fontsize=20)
        plt.imshow(mask_model_prediction)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask_model_prediction = np.where(original_size_prediction[:, :, layer] == img_label, 255, 0)
        plt.title("Model prediction mask", fontsize=20)
        plt.imshow(mask_model_prediction, cmap='gray')
        plt.axis('off');

    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer)

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…